In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer
!!pip install hf_transfer

['Collecting hf_transfer',
 '  Downloading hf_transfer-0.1.8-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.7 kB)',
 'Downloading hf_transfer-0.1.8-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)',
 '\x1b[?25l   \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.0/3.6 MB\x1b \x1b?\x1b eta \x1b-:--:--\x1b',
 '\x1b[2K   \x1b━\x1b\x1b╺\x1b\x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.1/3.6 MB\x1b \x1b3.5 MB/s\x1b eta \x1b0:00:01\x1b',
 '\x1b[2K   \x1b━━━━━━━━━━━━\x1b\x1b╸\x1b\x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b1.1/3.6 MB\x1b \x1b16.6 MB/s\x1b eta \x1b0:00:01\x1b',
 '\x1b[2K   \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b\x1b╸\x1b \x1b3.6/3.6 MB\x1b \x1b39.6 MB/s\x1b eta \x1b0:00:01\x1b',
 '\x1b[2K   \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b3.6/3.6 MB\x1b \x1b32.1 MB/s\x1b eta \x1b0:00:00\x1b',
 '\x1b[?25hInstalling collected packages: hf_transfer',
 'Successfully installed hf_transfer-0.1.8']

In [ ]:
import pandas as pd
feature_imp_ohe = pd.read_csv('/content/drive/MyDrive/Indian_bank_data/rf_importances.csv')
feature_imp_label = pd.read_csv('/content/drive/MyDrive/Indian_bank_data/rf_importances_label.csv')
train_ohe = pd.read_csv('/content/drive/MyDrive/Indian_bank_data/train_tree.csv')
train_ohe = train_ohe[~train_ohe["Approved_Flag"].isna()]
train_label = pd.read_csv('/content/drive/MyDrive/Indian_bank_data/train_label.csv')
train_label = train_label[~train_label["Approved_Flag"].isna()]
test_ohe = pd.read_csv('/content/drive/MyDrive/Indian_bank_data/test_tree.csv')
test_label = pd.read_csv('/content/drive/MyDrive/Indian_bank_data/test_label.csv')
test_ohe = test_ohe[~test_ohe["Approved_Flag"].isna()]
test_label = test_label[~test_label["Approved_Flag"].isna()]
test_ohe_100 = test_ohe.sample(n=100, random_state=42)
test_label_100 = test_label.sample(n=100, random_state=42)

In [ ]:
feature_description = pd.read_csv('/content/drive/MyDrive/Indian_bank_data/Capstone - Data Dictionary - Sheet1.csv')

In [ ]:
def mask_last_column_in_last_n_rows(df, n, sample_size=32):
    sampled_df = df.sample(n=sample_size, random_state=42) if len(df) >= sample_size else df.copy()

    formatted_text = "--- data ---\n"
    formatted_text += "|" + "|".join(sampled_df.columns) + "|\n"  # Header row with column names

    # List to store the actual masked values for the last n rows
    masked_values = []

    # Iterate over rows and apply masking for the last n rows
    for i, (_, row) in enumerate(sampled_df.iterrows(), start=1):
        row_values = list(row.values)
        if i > len(sampled_df) - n:  # Apply mask to the last n rows
            if n == 1:
                row_values[-1] = "<MASK>"
            else:
                mask_num = i - (len(sampled_df) - n)
                row_values[-1] = f"<MASK{mask_num}>"
            masked_values.append(row[row.index[-1]])  # Store the actual value being masked
        formatted_text += "|" + "|".join(str(value) for value in row_values) + "|\n"

    if n == 1:
        formatted_text += "\nPlease use the supplied data to predict the <MASK> Approved_Flag.\n"
    else:
        formatted_text += f"\nPlease use the supplied data to predict the <MASK{k}> Approved_Flag for k = 1,2,...,{n}.\n"

    # Add actual masked values separated by a comma
    formatted_text += "Answer: " + ", ".join(str(val) for val in masked_values)

    return formatted_text


In [ ]:
first_part = """You are an expert in the financial sector and banking industry.
Based on the credit information of individuals, please predict the Approval_Flag.
I will supply multiple instances with features and the corresponding label for your reference.
Please refer to the table below for detailed descriptions of the features and label:\n"""

In [ ]:
def generate_feature_label_descriptions(data_dict_df):

    feature_descriptions = "--- feature description ---\n"
    label_descriptions = "--- label description ---\n"

    # Process all rows except the last one as feature descriptions
    for _, row in data_dict_df.iloc[:-1].iterrows():
        feature_name = row['Feature']
        description = row['Feature Description']
        feature_descriptions += f"{feature_name}: {description}\n"

    # Treat the last row as the label description
    label_row = data_dict_df.iloc[-1]
    label_name = label_row['Feature']
    label_description = label_row['Feature Description']
    label_descriptions += f"{label_name}: {label_description}\n"

    # Combine both sections
    return feature_descriptions + label_descriptions



In [ ]:
# description_prompt = generate_feature_label_descriptions(feature_description)

In [ ]:
from huggingface_hub import login
login()  # Required for private or licensed models


**How to load the model**


1.   Regster for LLaMA on Meta website
2.   Download the model data using Hugging Face or Github (If you use hugginface, your directory is like the following: /content/LLaMA-2-7b/models--meta-llama--Llama-2-7b-hf/snapshots/01c7f73d771dfac7d292323805ebc428287df4f9. The 01c7... includes config.json)
3.   Download files in manage_ckpt in the Model github (or you can fork the github repo) and run the following LLAMA_MODEL_PATH="/content/LLaMA-2-7b/models--meta-llama--Llama-2-7b-hf/snapshots/01c7f73d771dfac7d292323805ebc428287df4f9"
CKPT_SAVE_DIR="/content"                # Directory to save the final model
MODEL_SIZE="7B"                         # Specify "7B" or "13B"

  !bash /content/recover_model.sh {dollarmark}LLAMA_MODEL_PATH {dollarmark}CKPT_SAVE_DIR {dollarmark}MODEL_SIZE, {dollarmark} = $

4.    Load model by the following recovered_model_path = "/content/LLaMA-2-GTL/7B"

  model = AutoModelForCausalLM.from_pretrained(recovered_model_path)

  tokenizer = AutoTokenizer.from_pretrained(recovered_model_path)



this is to download meta's original llama model. Skip, since we already have downloaded it once

In [ ]:
# recovered_model_path = "/content/drive/MyDrive/LLaMA-2-7b-checkpoint-GTL"
# model = AutoModelForCausalLM.from_pretrained(recovered_model_path)
# tokenizer = AutoTokenizer.from_pretrained(recovered_model_path)

# base_model_path = "/content/drive/MyDrive/LLaMA-2-13b"  # Define a path to save the model
# model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-13b-hf", cache_dir=base_model_path)
# tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-13b-hf", cache_dir=base_model_path)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

This is to download the GTL-enhanced model weight differences and recover the LLaMA-2-GTL checkpoint. Skip, since we already have downloaded it once

In [ ]:
# LLAMA_MODEL_PATH="/content/drive/MyDrive/LLaMA-2-7b/models--meta-llama--Llama-2-7b-hf/snapshots/01c7f73d771dfac7d292323805ebc428287df4f9"
# CKPT_SAVE_DIR="/content/drive/MyDrive/LLaMA-2-7b-checkpoint"                # Directory to save the final model
# MODEL_SIZE="7B"                         # Specify "7B" or "13B"

# LLAMA_MODEL_PATH="/content/drive/MyDrive/LLaMA-2-13b/models--meta-llama--Llama-2-13b-hf/snapshots/5c31dfb671ce7cfe2d7bb7c04375e44c55e815b1"
# CKPT_SAVE_DIR="/content/drive//MyDrive/LLaMA-2-13b-checkpoint"                # Directory to save the final model
# MODEL_SIZE="13B"                         # Specify "7B" or "13B"

# Run the recovery script
# !bash /content/drive/MyDrive/Industrial-Foundation-Models/scripts/manage_ckpt/recover_model.sh $LLAMA_MODEL_PATH $CKPT_SAVE_DIR $MODEL_SIZE

In [ ]:
# need this if u running quantized models
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
# # after pip install, need to restart colab session (not "Disconnect n delete runtime"), in order to load_in_4bit for next cell's recovery of model

In [ ]:
# from transformers import AutoModelForCausalLM, AutoTokenizer

# # Load the recovered model
recovered_model_path = "/content/drive/MyDrive/LLaMA-2-7b-checkpoint-GTL-Delta"
# recovered_model_path = "/content/drive/MyDrive/LLaMA-2-13b-checkpoint-GTL-Delta"

# # remove load_in_4bit=True if running original unquantized model
model = AutoModelForCausalLM.from_pretrained(recovered_model_path, load_in_8bit=True) #, load_in_4bit=True, device_map="auto"
tokenizer = AutoTokenizer.from_pretrained(recovered_model_path) #, device_map="auto"

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (no

In [ ]:
import random
# Function to create multiple sets of in-context example: input should be size of sample and number of set, with the dataframe, return with id of the set

def create_in_context_example_sets(df, sample_size, num_sets, target_column, proportions):
    example_sets = {}
    set_counter = 1

    # Generate sets for each specified proportion of class 1
    for prop_1 in proportions:
        # Calculate the number of rows for class 1 and class 0
        num_rows_class_1 = int(sample_size * prop_1)
        num_rows_class_0 = sample_size - num_rows_class_1  # Remaining for class 0

        # Creating multiple sets will ensure that we are performing a thorough experiment across multiple sets
        for _ in range(num_sets):
            # Sample rows for class 1 and class 0
            class_1_rows = df[df[target_column] == 1].sample(n=num_rows_class_1, random_state=random.randint(0, 1000))
            class_0_rows = df[df[target_column] == 0].sample(n=num_rows_class_0, random_state=random.randint(0, 1000))

            # Combine samples for each class to form a single set and shuffle to ensure that the class rows are not clustered
            sampled_df = pd.concat([class_1_rows, class_0_rows]).sample(frac=1, random_state=random.randint(0, 1000)).reset_index(drop=True)
            example_sets[f"Set_{set_counter}_Prop_{prop_1}"] = sampled_df
            set_counter += 1

    return example_sets



In [ ]:
t_annoy_prompt = """
The following is a table with features in the first few columns and the label in the last column.
I will supply multiple instances with features and the corresponding label for your reference.
"""

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate_in_context_performance(training_set, test_data, tokenizer, model, max_new_tokens=10, feature_description=feature_description, table_format="t-table", show_template=True):
    """
    Records the balance of the training set, and calculates accuracy, precision, recall, and F1 score for a set of in-context examples.
    If training_set is None, performs zero-shot evaluation.

    Parameters:
    - training_set (pd.DataFrame or None): The training dataset or None for zero-shot.
    - test_data (pd.DataFrame): The test dataset for evaluation.
    - tokenizer: The tokenizer to process text inputs.
    - model: The model for generating predictions.
    - max_new_tokens (int): The maximum number of new tokens for generation.
    - feature_description (pd.DataFrame or None): DataFrame with feature descriptions.

    Returns:
    - dict: A dictionary with balance of training set, accuracy, precision, recall, and f1 score.
    """
    # Record Training Balance
    description_prompt = generate_feature_label_descriptions(feature_description[feature_description["Feature"].isin(test_data.columns.to_list())])
    if table_format == "t-table":
      if training_set is not None:
          # Prepare in-context examples from the training set
          in_context_data = "\n".join([f"|{'|'.join(map(str, row))}|" for _, row in training_set.iterrows()])
          # Display column names and data rows with prompt
          in_context_examples = first_part + description_prompt + "--- data ---\n" + "|" + "|".join(test_data.columns) + in_context_data
      else:
          in_context_examples = first_part + description_prompt + "--- data ---\n" + "|" + "|".join(test_data.columns) + "|"
    else:
      in_context_examples = t_annoy_prompt
      if training_set is not None:
          in_context_data = "\n".join([f"|{'|'.join(map(str, row))}|" for _, row in training_set.iterrows()])
          in_context_examples += in_context_data
    # Prepare Test Prompts and Labels
    y_true = []
    y_pred = []
    non_numeric_count = 0
    start = show_template
    for _, row in test_data.iterrows():
        # Mask the label in the test row
        row_values = row.tolist()
        row_values[-1] = "<MASK>"  # Assuming the label is the last column
        masked_test_row = f"|{'|'.join(map(str, row_values))}|"

        # Combine in-context examples with the masked test row
        full_prompt = in_context_examples + "\n" + masked_test_row + "\n"
        full_prompt += "\nPlease use the supplied data to predict the <MASK> Approved_Flag.\nAnswer: " if table_format == "t-table" else "\nPlease use the supplied data to predict the <MASK> label.\nAnswer: "

        answer = row[-1]  # Store the ground truth label
        prompt_without_answer = full_prompt
        print("Prompt template:",prompt_without_answer)
        print("Answer template:",answer)
        if start:
          print("Prompt template:",prompt_without_answer)
          print("Answer template:",answer)
          start = False
        # Tokenize and generate answer
        inputs = tokenizer(prompt_without_answer, return_tensors="pt").to(device)
        if len(inputs) > 4096:
            print(f"Token size exceeded: {len(inputs)}")
            return
        input_ids = inputs['input_ids']

        # Generate predictions
        outputs = model.generate(
              input_ids=input_ids,
              attention_mask=inputs['attention_mask'],
              max_new_tokens=max_new_tokens
        )
        generated_answer = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)
        # print(generated_answer) # if u wanna troubleshoot and see what predictions are generated
        # Append true and predicted values
        y_true.append(answer)
        if len(generated_answer) > 0 and generated_answer[0].isdigit() and int(generated_answer[0]) in [0, 1]:
          y_pred.append(int(generated_answer[0]))
        else:
          non_numeric_count += 1
          if non_numeric_count > 15:
            break
          y_pred.append("invalid")
        del inputs, input_ids, outputs
        # Debugging output for intermediate results
        # print("Prompt without answer:\n", prompt_without_answer)
        # print("Groundtruth:", answer)
        # print("Generated answer:", generated_answer)
        # print("---")
    print("Num of non-numeric or invalid:", non_numeric_count)
    # Calculate Metrics
    pred = y_pred.copy()
    filtered_y_true = [yt for yt, yp in zip(y_true, y_pred) if isinstance(yp, int)]
    filtered_y_pred = [yp for yp in y_pred if isinstance(yp, int)]

    # Convert back to arrays if needed
    y_true = np.array(filtered_y_true)
    y_pred = np.array(filtered_y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division="warn")
    recall = recall_score(y_true, y_pred, zero_division="warn")
    f1 = f1_score(y_true, y_pred, zero_division="warn")

    # Combine Results
    results = {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1
    }

    return results, pred


In [ ]:
def select_top_features(feature_importance_df, top_n):
    top_features = feature_importance_df.nlargest(top_n, 'Importance')['Feature'].tolist()
    return top_features


In [ ]:
import pandas as pd

def run_experiments(feature_importance_df, test_data, tokenizer, model, training_set_df, sample_size, proportions, num_sets=3, feature_set_sizes=[5, 10, 20, 30, 40], run_multiple=True, table_format="t-table"):
    """
    Runs experiments with varying feature sets, training set sizes with controlled imbalances, and records model performance and stability.

    Parameters:
    - feature_importance_df (pd.DataFrame): DataFrame with 'Feature' and 'Importance' columns.
    - test_data (pd.DataFrame): Test dataset.
    - tokenizer: Tokenizer for prompt processing.
    - model: Model for generating predictions.
    - training_set_df (pd.DataFrame): Original training dataset to sample in-context examples from.
    - sample_size (int): Number of examples per set for the training data.
    - proportions (list): List of proportions for class 1 in training sets to control imbalance.
    - num_sets (int): Number of sets to generate per training set size and feature set size.
    - feature_set_sizes (list): List of feature set sizes to evaluate.

    Returns:
    - tuple: A DataFrame recording results for each experiment and a dictionary storing in-context sets.
    """
    results = []
    stored_in_context_sets = {}

    for top_n in feature_set_sizes:
        # Select top features based on importance
        top_features = select_top_features(feature_importance_df, top_n)
        training_set_df_subset = training_set_df[top_features + ['Approved_Flag']]  # Subset training data
        test_data_subset = test_data[top_features + ['Approved_Flag']]  # Subset test data with selected features
        print(f"\nStarting experiments for top {top_n} features...")

        for prop_1 in proportions:
            print(f"\nTesting with class 1 proportion: {prop_1}")
            # Generate sets of in-context examples for the current proportion of class 1
            in_context_sets = create_in_context_example_sets(training_set_df_subset, sample_size, num_sets, 'Approved_Flag', [prop_1])

            # Store each in-context set in the dictionary with a unique identifier
            for set_id, in_context_examples in in_context_sets.items():
                unique_id = f"Features_{top_n}_Prop_{prop_1}_Set_{set_id}"
                stored_in_context_sets[unique_id] = in_context_examples  # Store the in-context set
                print(f"\nEvaluating Set ID: {set_id}")

                y_pred_all = []  # To store predictions for stability check
                times = 4 if run_multiple else 2
                for run_num in range(1, times):  # 3 runs per set for stability assessment
                    # Evaluate using in-context examples from `in_context_examples`
                    results_dict, y_pred = evaluate_in_context_performance(in_context_examples, test_data_subset, tokenizer, model, table_format=table_format)

                    # Record metrics
                    run_results = {
                        "Num Features": top_n,
                        "Sample Size": sample_size,
                        "Class 1 Proportion": prop_1,
                        "Set ID": set_id,
                        "Run Number": run_num,
                        "Accuracy": results_dict["Accuracy"],
                        "Precision": results_dict["Precision"],
                        "Recall": results_dict["Recall"],
                        "F1 Score": results_dict["F1 Score"],
                        "Prediction": y_pred
                    }
                    results.append(run_results)

                    # Print intermediate results for each run
                    print(f"Intermediate results for Run {run_num}: {run_results}")

    # Convert results to DataFrame for easy viewing and analysis
    results_df = pd.DataFrame(results)
    print("\nAll experiments completed.")
    return results_df, stored_in_context_sets

# Example usage:
# proportions = [0.2, 0.5, 0.8]  # Different proportions of class 1 to create various imbalances
# experiment_results, in_context_sets_dict = run_experiments(feature_importance_df, test_data, tokenizer, model, training_set_df, sample_size=50, proportions=proportions)
# print(experiment_results)
# print("Stored in-context sets:", in_context_sets_dict.keys())


Zero-shot (feature nums: 5, 10, 20, 30, 40) (ohe). Ignore this example and skip down to function 'save_in_context_sets_to_csv' and beyond

Example

In [ ]:
# proportions = [0.1, 0.3, 0.5]  # Different proportions of class 1 to create various imbalances
# experiment_results = run_experiments(feature_imp_ohe, test_ohe_100, tokenizer, model, train_ohe, sample_size=0, proportions=proportions)
# print(experiment_results)

Intermediate results for Run 1: {'Num Features': 5, 'Sample Size': 0, 'Class 1 Proportion': 0.1, 'Set ID': 'Set_1_Prop_0.1', 'Run Number': 1, 'Accuracy': 0.69, 'Precision': 0.796547619047619, 'Recall': 0.69, 'F1 Score': 0.7320846755637368}


Intermediate results for Run 2: {'Num Features': 5, 'Sample Size': 0, 'Class 1 Proportion': 0.1, 'Set ID': 'Set_1_Prop_0.1', 'Run Number': 2, 'Accuracy': 0.61, 'Precision': 0.7666577540106952, 'Recall': 0.61, 'F1 Score': 0.6703629536921152}

Intermediate results for Run 3: {'Num Features': 5, 'Sample Size': 0, 'Class 1 Proportion': 0.1, 'Set ID': 'Set_1_Prop_0.1', 'Run Number': 3, 'Accuracy': 0.69, 'Precision': 0.796547619047619, 'Recall': 0.69, 'F1 Score': 0.7320846755637368}
Instability count for Set Set_1_Prop_0.1 with proportion 0.1: 0

In [ ]:
import os

def save_in_context_sets_to_csv(stored_in_context_sets, filename, output_dir="/content/drive/MyDrive/Indian_bank_data"):
    """
    Saves each in-context set in stored_in_context_sets as a separate CSV file, with each set having a unique column name.

    Parameters:
    - stored_in_context_sets (dict): Dictionary containing in-context sets with unique IDs as keys.
    - filename (str): The base filename for the CSV files.
    - output_dir (str): Directory where CSV files will be saved. Defaults to 'in_context_sets'.
    """
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Prepare a combined DataFrame to save all sets in a single CSV file
    combined_df = pd.DataFrame()

    # Iterate over each in-context set
    for set_id, in_context_df in stored_in_context_sets.items():
        # Add the set_id as a column name
        in_context_df.columns = [f"{set_id}_{col}" for col in in_context_df.columns]

        # Concatenate with combined_df
        combined_df = pd.concat([combined_df, in_context_df], axis=1)

        # List of columns to check and replace 'invalid'
        metric_columns_to_replace_invalids = ['Accuracy', 'Precision', 'Recall', 'F1_Score', 'PR_AUC', 'ROC_AUC']
        # Replace 'invalid' with 0 in the specified columns
        combined_df[metric_columns_to_replace_invalids].replace('invalid', 0, inplace=True)

    # Save the combined DataFrame as a single CSV with the specified filename
    file_path = os.path.join(output_dir, f"{filename}.csv")
    combined_df.to_csv(file_path, index=False)
    print(f"Saved all in-context sets to {file_path}")

# Example usage after running run_experiments:
# save_in_context_sets_to_csv(in_context_sets_dict, filename="all_in_context_sets")


Example loop for t-table and t-annony

Run this. Run based on the...

- sample_sizes
- t-table/t-annony format

that you need to run

In [ ]:
proportions = [0.1, 0.3, 0.5]  # Different proportions of class 1 to create various imbalances
sample_sizes = [0, 8, 16, 32, 48, 64]  # Different sample sizes to iterate over

# Loop over each sample size
for sample_size in sample_sizes:
    experiment_results, stored_in_context_sets = run_experiments(
        feature_imp_ohe, test_ohe_100, tokenizer, model, train_ohe,
        sample_size=sample_size, proportions=proportions,
        feature_set_sizes=[5, 10, 20, 30, 40], num_sets=15,
        run_multiple=False, table_format="t-table"
    )

    # Save results to CSV
    file_name = f"experiments_result_t_table_show_{sample_size}_Nov21.csv"
    experiment_results.to_csv(file_name, index=False)

    print(f"Results for sample_size={sample_size} saved as {file_name}")

<ipython-input-37-da7f33bae405>:49: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  answer = row[-1]  # Store the ground truth label
<ipython-input-37-da7f33bae405>:49: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  answer = row[-1]  # Store the ground truth label
<ipython-input-37-da7f33bae405>:49: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  answer = row[-1]  # Store the ground truth label


KeyboardInterrupt: 

In [ ]:
import pandas as pd

proportions = [0.1, 0.3, 0.5]  # Different proportions of class 1 to create various imbalances
sample_sizes = [0, 8, 16, 32, 64, 128]  # Different sample sizes to iterate over

# Loop over each sample size
for sample_size in sample_sizes:
    experiment_results, stored_in_context_sets = run_experiments(
        feature_imp_ohe, test_ohe_100, tokenizer, model, train_ohe,
        sample_size=sample_size, proportions=proportions,
        feature_set_sizes=[5, 10, 20, 30, 40], num_sets=1,
        run_multiple=False, table_format="t-annony"
    )

    # Save results to CSV
    file_name = f"experiments_result_t_annoy_show_{sample_size}.csv"
    experiment_results.to_csv(file_name, index=False)

    print(f"Results for sample_size={sample_size} saved as {file_name}")
